# Import libraries

In [4]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [ ]:
import pandas as pd
import os

In [2]:
data = pd.read_json(os.path.join("..", "data", "test.json"))
data 

,text
0,As a consequence of the global COVID-19 pandem...
1,According to current live statistics at the ti...


# Prepocessing

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabrielduong/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
def preprocess_and_vectorize(df, text_column='text',  max_df=0.85, min_df=0.01):
    """
    Preprocess text data for topic modeling and return TF-IDF matrix.
    
    Parameters:
        df (pd.DataFrame): DataFrame with a column of raw text.
        text_column (str): Name of the column with text data.
        min_df (int): Ignore terms with doc freq < min_df.
        max_df (float): Ignore terms with doc freq > max_df fraction of docs.
    
    Returns:
        pd.DataFrame: TF-IDF matrix (documents x words)
    """
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    
    def clean_text(text):
        # Lowercase
        text = text.lower()
        # Remove punctuation and digits
        text = re.sub(r'[^a-z\s]', '', text)
        # Tokenize
        tokens = text.split()
        # Remove stopwords and stem
        tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    
    # Apply cleaning
    cleaned_corpus = df[text_column].astype(str).apply(clean_text)
    
    # Vectorization
    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    tfidf_matrix = vectorizer.fit_transform(cleaned_corpus)
    
    # Convert to DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    
    return tfidf_df


In [13]:
tfidf_df = preprocess_and_vectorize(data, text_column='text')
tfidf_df.head()

,abil,abl,academi,acceler,access,account,accur,accuraci,action,actual,...,wherea,whether,wide,window,work,world,worldwid,would,write,wtft
0,0.05712,0.02856,0.02856,0.02856,0.05712,0.000000,0.05712,0.114239,0.000000,0.02856,...,0.02856,0.02856,0.02856,0.02856,0.000000,0.000000,0.000000,0.114239,0.000000,0.257038
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.042409,0.00000,0.000000,0.042409,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.084819,0.042409,0.084819,0.000000,0.042409,0.000000
